In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
import scipy.interpolate

Get the value of ah from luminosity scaling, using methodology as in Krause et al. 2015

In [2]:
# Set up a function for the comoving distance

def com(z_, OmC, OmB, HH0, Nnu):
    """ Gets the comoving distance in units of Mpc/h at a given redshift, z_ (assuming the cosmology defined in the params file). """
    
    OmR	=	2.47*10**(-5)/(HH0/100.)**2
    OmN	=	Nnu*(7./8.)*(4./11.)**(4./3.)*OmR
    OmL = 1. - OmC - OmB - OmR - OmN
    c=2.99792458*10**(8)
    H0	=	10**(5)/c
    
    def chi_int(z):
        return 1. / (H0 * ( (OmC+OmB)*(1+z)**3 + OmL + (OmR+OmN) * (1+z)**4 )**(0.5))

    if hasattr(z_, "__len__"):
        chi=np.zeros((len(z_)))
        for zi in range(0,len(z_)):
            #print "zi in com=", zi
            chi[zi] = scipy.integrate.quad(chi_int,0,z_[zi])[0]
    else:
        chi = scipy.integrate.quad(chi_int, 0, z_)[0]

    return chi


def get_dNdzL(zvec, survey):
    """ Imports the lens redshift distribution from file, normalizes, interpolates, and outputs at the z vector that's passed."""

    if (survey == 'SDSS'):
        import params as pa
    elif (survey == 'LSST_DESI'):
        import params_LSST_DESI as pa
    else:
        print "We don't have support for that survey yet; exiting."
        exit()
    
    z, dNdz = np.loadtxt('./txtfiles/'+pa.dNdzL_file, unpack=True)

    interpolation = scipy.interpolate.interp1d(z, dNdz)

    # Create a well-sampled redshift vector to make sure we can get the normalization without numerical problems
    z_highres = np.linspace(z[0], z[-1], 1000)

    dNdz_getnorm = interpolation(z_highres)

    norm = scipy.integrate.simps(dNdz_getnorm, z_highres)

    if ((zvec[0]>=z[0]) and (zvec[-1]<=z[-1])):
        dNdz_return = interpolation(zvec)
    else:
        print "You have asked for dN/dzl at redshifts out of the known range."
        exit()

    return dNdz_return / norm

In [3]:
# Set up the Schechter luminosity function parameters from Krause et al. 2015

mlim_1 = 22.; mlim_2 = 27.5; 
Mp = -22.
Lp = 10.**(-0.4*(Mp-Mp))

# Power law parameters
alpha_h = 0.081; beta_h = 2.1
A_0 = 4.9; beta = 1.30;

# We are using parameters from Krause et al. 2015, red galaxies
#Mr_s = -20.34; Q = 1.8; alpha = -0.57; phi_0 = 1.1 * 10**(-2); P = -1.2 # This set is all from GAMA
Mr_s = -20.34; Q = 1.2; alpha = -0.57; phi_0 = 1.1 * 10**(-2); P = -1.15 # This set uses Q & P scaled from DEEP2


# Cosmological parameters
Nnu	=	3.046    # Massless neutrinos
HH0 = 67.26 
OmR	=	2.47*10**(-5)/(HH0/100.)**2
OmN	=	Nnu*(7./8.)*(4./11.)**(4./3.)*OmR
OmB	=	0.02222/(HH0/100.)**2 
OmC	=	0.1199/(HH0/100.)**2 
OmM=  OmB+OmC
A_s	=	2.2 * 10**(-9)
n_s	=	0.9652


In [4]:
# Compute phi*(z) and M*(z) / L*(z), where z is the lens redshift 
zL1 = np.linspace(0.16, 0.36, 200); zL2 = np.linspace(0.025, 1.175, 200)

phi_s_1 = phi_0 * 10**(0.4 * P*zL1); 
phi_s_2 = phi_0 * 10**(0.4 * P*zL2); 

Ms_1 = Mr_s - Q * ( zL1 - 0.1 ); Ms_2 = Mr_s - Q * ( zL2 - 0.1)
Ls_1 = 10.**(-0.4 *(Ms_1-Mp)); Ls_2 = 10.**(-0.4 *(Ms_2-Mp))

In [5]:
# Import the k correction and EC correction from Poggianti (assuming elliptical galaxies)
(z_k, kcorr, x,x,x) = np.loadtxt('./txtfiles/kcorr.dat', unpack=True)
(z_e, ecorr, x,x,x) = np.loadtxt('./txtfiles/ecorr.dat', unpack=True)

kcorr_interp = scipy.interpolate.interp1d(z_k, kcorr)
ecorr_interp = scipy.interpolate.interp1d(z_e, ecorr)

kcorr_1 = kcorr_interp(zL1)
ecorr_1 = ecorr_interp(zL1)
kcorr_2 = kcorr_interp(zL2)
ecorr_2 = ecorr_interp(zL2)

In [6]:
# Get the absolute magnitude that corresponds to the limiting apparent magnitude
dl_1 = com(zL1, OmC, OmB, HH0, Nnu) * (1. + zL1)
dl_2 = com(zL2, OmC, OmB, HH0, Nnu) * (1. + zL2)

print "sdss=", com(0.28, OmC, OmB, HH0, Nnu) * (1. + 0.28)
print "lsst=", com(0.77, OmC, OmB, HH0, Nnu) * (1. + 0.77)

Mlim_1 = mlim_1 - (5. * np.log10(dl_1) + 25. + kcorr_1 + ecorr_1); Llim_1 = 10.**(-0.4 * (Mlim_1-Mp))
print "Mlim 1=", Mlim_1
Mlim_2 = mlim_2 - (5. * np.log10(dl_2) + 25. + kcorr_2 + ecorr_2); Llim_2 = 10.**(-0.4 * (Mlim_2-Mp))
#print "Mlim 2=", Mlim_2


sdss= 1001.06847705
lsst= 3327.99294587
Mlim 1= [-16.62541756 -16.64106588 -16.65662773 -16.67210417 -16.68749623
 -16.70280492 -16.71803123 -16.73317613 -16.7482406  -16.76322556
 -16.77813194 -16.79296065 -16.80771258 -16.82238861 -16.8369896
 -16.85151639 -16.86596982 -16.88035071 -16.89465986 -16.90889806
 -16.92306608 -16.9371647  -16.95119466 -16.96515671 -16.97905156
 -16.99287994 -17.00664255 -17.02034009 -17.03397322 -17.04754263
 -17.06104898 -17.07449291 -17.08787506 -17.10119607 -17.11445654
 -17.12765711 -17.14079835 -17.15388087 -17.16690525 -17.17987205
 -17.19268638 -17.20506236 -17.21738243 -17.22964715 -17.24185704
 -17.25401263 -17.26611444 -17.27816297 -17.29015873 -17.30210222
 -17.31399391 -17.3258343  -17.33762385 -17.34936304 -17.36105232
 -17.37269215 -17.38428297 -17.39582524 -17.40731937 -17.41876582
 -17.43016498 -17.4415173  -17.45282317 -17.46408301 -17.47529722
 -17.48646619 -17.49759032 -17.50866998 -17.51970557 -17.53069745
 -17.541646   -17.55255159 -1

In [7]:
# Get the luminosity vectors over which to integrate
# For each case, there will be a list of these, one for each redshift,
# because the limiting values if z-dependent

L_1 = [0] * len(zL1)
for zi in range(0,len(zL1)):
    L_1[zi] = scipy.logspace(np.log10(Llim_1[zi]), 2, 1000)
L_2 = [0]*len(zL2)
for zi in range(0,len(zL2)):
    L_2[zi] = scipy.logspace(np.log10(Llim_2[zi]), 2, 1000)

# Now get phi(L,z), where this exists for each z because the lenghts of the L vectors are different.
phi_func_1 = [0]*len(zL1)
for zi in range(0,len(zL1)):
    phi_func_1[zi]= np.zeros(len(L_1[zi]))
    for li in range(0,len(L_1[zi])):
        phi_func_1[zi][li] = phi_s_1[zi] * (L_1[zi][li] / Ls_1[zi]) ** (alpha) * np.exp(- L_1[zi][li] / Ls_1[zi])

phi_func_2 = [0]*len(zL2)
for zi in range(0,len(zL2)):
    phi_func_2[zi]= np.zeros(len(L_2[zi]))
    for li in range(0,len(L_2[zi])):
        phi_func_2[zi][li] = phi_s_2[zi] * (L_2[zi][li] / Ls_2[zi]) ** (alpha) * np.exp(- L_2[zi][li] / Ls_2[zi])        
       

In [8]:
# Get the numerator and denom (norm) of the required averages in luminosity, still as functions of zL

norm_1 = np.zeros(len(zL1)); num_ah_1 = np.zeros(len(zL1)); num_Ai_1 = np.zeros(len(zL1)) 
avgL_1 = np.zeros(len(zL1))
for zi in range(len(zL1)):
    norm_1[zi] = scipy.integrate.simps(phi_func_1[zi], L_1[zi])
    num_ah_1[zi] = scipy.integrate.simps(phi_func_1[zi] * alpha_h * (L_1[zi] / Lp)**(beta_h), L_1[zi])
    num_Ai_1[zi] = scipy.integrate.simps(phi_func_1[zi] * A_0 * (L_1[zi] / Lp)**(beta), L_1[zi])
    avgL_1[zi] = scipy.integrate.simps(phi_func_1[zi] * L_1[zi] / Lp, L_1[zi])
    
norm_2 = np.zeros(len(zL2)); num_ah_2 = np.zeros(len(zL2)); num_Ai_2 = np.zeros(len(zL2))
avgL_2=np.zeros(len(zL2))
for zi in range(len(zL2)):
    norm_2[zi] = scipy.integrate.simps(phi_func_2[zi], L_2[zi])
    num_ah_2[zi] = scipy.integrate.simps(phi_func_2[zi] * alpha_h * (L_2[zi] / Lp)**(beta_h), L_2[zi])
    num_Ai_2[zi] = scipy.integrate.simps(phi_func_2[zi] * A_0 * (L_2[zi] / Lp)**(beta), L_2[zi])
    avgL_2[zi] = scipy.integrate.simps(phi_func_2[zi] * L_2[zi] / Lp, L_2[zi])


In [9]:
# Get the lens galaxy distributions, over which we will integrate z
dNdzl_1 = get_dNdzL(zL1, 'SDSS')
dNdzl_2 = get_dNdzL(zL2, 'LSST_DESI')

# Test what happens when we revert to a zeff by using a narrow Gaussian
#sig_small = 0.01
#dNdzl_1 = 1./np.sqrt(2. * np.pi) / sig_small * np.exp(- (zL1 - 0.28)**2 / (2. * sig_small**2))
#dNdzl_2 = 1./np.sqrt(2. * np.pi) / sig_small * np.exp(- (zL2 - 0.77)**2 / (2. * sig_small**2))

# Get the final amplitude values and the average luminosities for each case:
ah_1 = scipy.integrate.simps(num_ah_1 / norm_1 * dNdzl_1, zL1)
Ai_1 = scipy.integrate.simps(num_Ai_1 / norm_1 * dNdzl_1, zL1)
avgL_1 = scipy.integrate.simps(avgL_1 / norm_1 * dNdzl_1, zL1)
ah_2 = scipy.integrate.simps(num_ah_2 / norm_2 * dNdzl_2, zL2)
Ai_2 = scipy.integrate.simps(num_Ai_2 / norm_2 * dNdzl_2, zL2)
avgL_2 = scipy.integrate.simps(avgL_2 / norm_2 * dNdzl_2, zL2)

print "###################################################"
print "For SDSS:"
print "ah =", ah_1, "Ai=", Ai_1, "Average L=", avgL_1
print "###################################################"
print "For LSST + DESI"
print "ah =", ah_2, "Ai=", Ai_2, "Average L=", avgL_2


###################################################
For SDSS:
ah = 0.00508024109555 Ai= 0.600388324123 Average L= 0.175014469581
###################################################
For LSST + DESI
ah = 0.0125376550036 Ai= 0.928226346412 Average L= 0.230261229935
